In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV

# Define parameter grid (smaller for speed)
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Instantiate the Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=42)

grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1, verbose=1)


# fitting the model for grid search
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy', verbose=1)

In [15]:
from sklearn.model_selection import cross_val_predict

# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid_search.best_estimator_, X, y, cv=5)

In [16]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [18]:
print("Best Parameters:", grid_search.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}
The f1_macro: 0.8376247189790194
The confusion Matrix:
 [[224  33]
 [ 32 111]]
The report:
               precision    recall  f1-score   support

           0       0.88      0.87      0.87       257
           1       0.77      0.78      0.77       143

    accuracy                           0.84       400
   macro avg       0.82      0.82      0.82       400
weighted avg       0.84      0.84      0.84       400



In [20]:
# Save Best Model
import pickle
filename = "SVC_CV_best_model.sav"
pickle.dump(grid_search.best_estimator_, open(filename, 'wb'))

In [22]:
# CV Results table
results_df = pd.DataFrame(grid_search.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005926,0.000966,0.003246,0.000227,gini,None,1,2,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.897196,0.841121,0.820755,0.853024,0.032322,53
1,0.004195,0.000036,0.002723,0.000051,gini,None,1,5,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.869159,0.859813,0.839623,0.856198,0.012326,51
2,0.004843,0.000868,0.002926,0.000065,gini,None,1,10,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.915888,0.887850,0.849057,0.884265,0.027401,31
3,0.004600,0.000736,0.002960,0.000111,gini,None,2,2,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.915888,0.869159,0.849057,0.878034,0.027996,44
4,0.004077,0.000053,0.003547,0.001036,gini,None,2,5,"{'criterion': 'gini', 'max_depth': None, 'min_...",0.915888,0.869159,0.849057,0.878034,0.027996,44
